In [1]:
import os
import numpy as np

Image loading function

In [2]:
def Load_CIFAR10_batch(file):
    import _pickle as cPickle
    fileopen = open(file , "rb")
    dict = cPickle.load(fileopen,encoding="latin1")
    fileopen.close
    
    return dict['data'].reshape(-1, 32, 32, 3), dict['labels']



In [3]:
print("Loading................")
batch_fns = [os.path.join(r"C:\Users\ravul\Downloads\cifar-10-batches-py", "data_batch_" + str(i)) for i in range(1,6)]
data_Batches = [Load_CIFAR10_batch(fn) for fn in batch_fns]

Loading................


Data Stacking

In [4]:
data_all = np.vstack([data_Batches[i][0] for i in range(len(data_Batches))]).astype('float')
labels_all = np.vstack([data_Batches[i][1] for i in range(len(data_Batches))]).flatten()

split the data of 50000 samples in 92:8 ratio, 4000 samples are used for the train and test sets for this clasification
StratifiedShuffleSplit is used to split dataset, It splits  the data by taking equal number of samples each class in a random manner 

In [5]:
# spliting the whole data set into 92:8 
seed = 7

from sklearn.model_selection import StratifiedShuffleSplit

datasplit = StratifiedShuffleSplit(1, test_size=0.08, random_state = seed) 

print(datasplit.get_n_splits(data_all, labels_all))

for train_index, test_index in datasplit.split(data_all, labels_all):
    split_data_92 , split_data_8 = data_all[train_index], data_all[test_index]
    split_label_92, split_label_8 = labels_all[train_index], labels_all[test_index]


1


4000 samples in trun divided into 7:3 ratio 

In [6]:
train_test_split = StratifiedShuffleSplit(1, test_size=0.3, random_state=seed)

print(train_test_split.get_n_splits(split_data_8, split_label_8))

for train_index, test_index in train_test_split.split(split_data_8, split_label_8):
    train_data_70, test_data_30 = split_data_8[train_index], split_data_8[test_index]
    train_label_70, test_label_30 = split_label_8[train_index], split_label_8[test_index]
    
train_data = train_data_70
test_data  = test_data_30
train_label = train_label_70
test_label  = test_label_30

print(train_data.shape, test_data.shape, train_label.shape, test_label.shape)

1
(2800, 32, 32, 3) (1200, 32, 32, 3) (2800,) (1200,)


Image preprocessing

In [7]:
# Normalizing images

def Normalize(data, eps=1e-8):
    
    data -= data.mean(axis=(1,2,3), keepdims=True)
    std = np.sqrt(data.var(axis=(1,2,3), ddof=1, keepdims=True))
    std[std<eps] = 1
    
    data /= std
    
    return(data)


train_data = Normalize(train_data)
test_data = Normalize(test_data)

print(train_data.shape, test_data.shape)

(2800, 32, 32, 3) (1200, 32, 32, 3)


In [10]:
# Whitening ZCA zero-phase component analysis.

train_data_flat = train_data.reshape(train_data.shape[0], -1).T
test_data_flat = test_data.reshape(test_data.shape[0], -1).T
print(train_data_flat.shape, test_data_flat.shape)
train_data_flat_T =  train_data_flat.T
test_data_flat_T = test_data_flat.T
print(train_data_flat_T.shape, test_data_flat_T.shape)


(3072, 2800) (3072, 1200)
(2800, 3072) (1200, 3072)


In [11]:
# PCA - principal component analysis dimensionality reduction

from sklearn.decomposition import PCA

train_data_pca = PCA(n_components=train_data_flat.shape[1]).fit_transform(train_data_flat)
test_data_pca = PCA(n_components=test_data_flat.shape[1]).fit_transform(test_data_flat)

train_data_pca = train_data_pca.T
test_data_pca =  test_data_pca.T


In [12]:
# SVD sinfular value decompostion (Dimensionality reduction )

print("Train data\n",train_data.shape,"\nTestdata\n", test_data.shape)

from skimage import color 

def svdFeatures(input_data):
    svd_array_inputdata = []
    size = input_data.shape[0]
    
    for i in range(0,size):
        img = color.rgb2gray(input_data[i])
        
        U, s, V = np.linalg.svd(img, full_matrices = False)
        
        S = [s[i] for i in range(30)]
        svd_array_inputdata.append(S)
        svd_Matrix_inputdata = np.matrix(svd_array_inputdata)
        
    return(svd_Matrix_inputdata)

#Apply SVD for train and test data

train_data_svd = svdFeatures(train_data)
test_data_svd = svdFeatures(test_data)

print(train_data_svd.shape, test_data_svd.shape)
    

Train data
 (2800, 32, 32, 3) 
Testdata
 (1200, 32, 32, 3)
(2800, 30) (1200, 30)


Classifier

Steps for the classification

- Initialize the classifier
- Train the classifier (All classifiers in the scikit-learn uses a fit(X,y) method to fit the model training for the given train data X and train label y)
- Predict the target (Given an unlabeled observation X , the predict(X) returns the predicted label y)
- Evaluate the classifier (The score(X,y) returns the score for the given test data X and test label y)

In [14]:
# SVM support vector machine

from sklearn import svm

classifier = svm.SVC(gamma=.001, probability=True)

classifier.fit(train_data_flat_T, train_label) # After being fitted, the model can then be used to predict the output 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
# predict and score

predicted  =  classifier.predict(test_data_flat_T)
print("Prediction \n", predicted, predicted.shape)

score = classifier.score(test_data_flat_T, test_label)
print("Score = ",score)

Prediction 
 [3 7 6 ... 4 8 8] (1200,)
Score =  0.38666666666666666
